# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
!pip install pyproj
!pip install cartopy
!pip install geoviews

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key

from api_keys import geoapify_key

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,293.73,82,8,11.12,PN,1696190710
1,1,thompson,55.7435,-97.8558,278.24,93,100,3.09,CA,1696190566
2,2,uliastay,47.7417,96.8444,274.90,86,28,1.94,MN,1696190710
3,3,happy valley-goose bay,53.3168,-60.3315,286.32,47,75,12.35,CA,1696190711
4,4,vilyuysk,63.7553,121.6247,278.71,83,100,1.79,RU,1696190711


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
%%capture --no-display

# Configure the map plot
import hvplot.pandas
import pyproj

map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = (city_data_df["Max Temp"] > 20) & (city_data_df["Humidity"] > 50) & (city_data_df["Cloudiness"] == 0)

ideal_cities_df = city_data_df[ideal_conditions]
# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,port elizabeth,-33.9180,25.5701,289.31,73,0,5.14,ZA,1696190492
30,30,bethel,41.3712,-73.4140,299.49,67,0,0.89,US,1696190458
44,44,bredasdorp,-34.5322,20.0403,284.57,83,0,2.91,ZA,1696190717
48,48,anadyr,64.7500,177.4833,268.43,93,0,1.00,RU,1696190717
52,52,albany,42.6001,-73.9662,299.45,62,0,0.89,US,1696190718
...,...,...,...,...,...,...,...,...,...,...
591,591,colonia,50.9333,6.9500,293.05,75,0,2.06,DE,1696190526
592,592,cole harbour,44.6724,-63.4751,293.81,80,0,4.12,CA,1696190785
593,593,valderice,38.0401,12.6151,294.70,74,0,3.95,IT,1696190785
596,596,kristiansund,63.1115,7.7320,284.18,87,0,2.57,NO,1696190785


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
12,port elizabeth,ZA,-33.9180,25.5701,73,
30,bethel,US,41.3712,-73.4140,67,
44,bredasdorp,ZA,-34.5322,20.0403,83,
48,anadyr,RU,64.7500,177.4833,93,
52,albany,US,42.6001,-73.9662,62,
...,...,...,...,...,...,...
591,colonia,DE,50.9333,6.9500,75,
592,cole harbour,CA,44.6724,-63.4751,80,
593,valderice,IT,38.0401,12.6151,74,
596,kristiansund,NO,63.1115,7.7320,87,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 20000  

params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat, lng = row["Lat"], row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params).json()
    print (response)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Waterford Hotel', 'country': 'South Africa', 'country_code': 'za', 'state': 'Eastern Cape', 'county': 'Nelson Mandela Bay Metropolitan Municipality', 'city': 'Gqeberha', 'postcode': '7162', 'district': 'Steytler', 'suburb': 'Nelson Mandela Bay Ward 11', 'street': 'Durban Road', 'lon': 25.5777512, 'lat': -33.9320089, 'state_code': 'EC', 'formatted': 'Waterford Hotel, Durban Road, Nelson Mandela Bay Ward 11, Gqeberha, 7162, South Africa', 'address_line1': 'Waterford Hotel', 'address_line2': 'Durban Road, Nelson Mandela Bay Ward 11, Gqeberha, 7162, South Africa', 'categories': ['accommodation', 'accommodation.hotel'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Waterford Hotel', 'osm_id': 1292374697, 'tourism': 'hotel

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Hampton Inn Danbury', 'country': 'United States', 'country_code': 'us', 'state': 'Connecticut', 'county': 'Western Connecticut Planning Region', 'city': 'Danbury', 'hamlet': 'Beaverbrook', 'postcode': '06801', 'street': 'Industrial Plaza Road', 'lon': -73.41135845863734, 'lat': 41.409656749999996, 'state_code': 'CT', 'formatted': 'Hampton Inn Danbury, Industrial Plaza Road, Danbury, CT 06801, United States of America', 'address_line1': 'Hampton Inn Danbury', 'address_line2': 'Industrial Plaza Road, Danbury, CT 06801, United States of America', 'categories': ['accommodation', 'accommodation.hotel', 'building', 'building.accommodation'], 'details': ['details'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Hampton Inn Danbury', 'brand': 'Hampton', 'osm_id

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Victoria Hotel', 'country': 'South Africa', 'country_code': 'za', 'state': 'Western Cape', 'county': 'Overberg District Municipality', 'city': 'Cape Agulhas Local Municipality', 'town': 'Bredasdorp', 'postcode': '7280', 'suburb': 'Cape Agulhas Ward 4', 'street': 'Church Street', 'housenumber': '10', 'lon': 20.0395361, 'lat': -34.5339684, 'state_code': 'WC', 'formatted': 'Victoria Hotel, 10 Church Street, Cape Agulhas Ward 4, Cape Agulhas Local Municipality, 7280, South Africa', 'address_line1': 'Victoria Hotel', 'address_line2': '10 Church Street, Cape Agulhas Ward 4, Cape Agulhas Local Municipality, 7280, South Africa', 'categories': ['accommodation', 'accommodation.hotel'], 'details': ['details', 'details.contact'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw':

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Tru', 'country': 'United States', 'country_code': 'us', 'state': 'New York', 'county': 'Albany County', 'city': 'Westmere', 'hamlet': 'Dunes', 'postcode': '12203', 'street': 'Western Avenue', 'housenumber': '1651', 'lon': -73.85295363780244, 'lat': 42.687101, 'state_code': 'NY', 'formatted': 'Tru, 1651 Western Avenue, Westmere, NY 12203, United States of America', 'address_line1': 'Tru', 'address_line2': '1651 Western Avenue, Westmere, NY 12203, United States of America', 'categories': ['accommodation', 'accommodation.hotel', 'building', 'building.accommodation'], 'details': ['details', 'details.building'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Tru', 'brand': 'Tru', 'osm_id': 701327863, 'tourism': 'hotel', 'website': 'https://www.hilton.com/en/h

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Dreamlan Golf Hotel \\ Drimlend Otel', 'country': 'Azerbaijan', 'country_code': 'az', 'state': 'Baku Ekonomic Zone', 'city': 'Baku City', 'postcode': '1089', 'district': 'Surakhany Raion', 'street': 'Qala - Hövsan yolu', 'lon': 50.0966544, 'lat': 40.4005187, 'formatted': 'Dreamlan Golf Hotel \\ Drimlend Otel, Qala - Hövsan yolu, 1089 Baku City, Azerbaijan', 'address_line1': 'Dreamlan Golf Hotel \\ Drimlend Otel', 'address_line2': 'Qala - Hövsan yolu, 1089 Baku City, Azerbaijan', 'categories': ['accommodation', 'accommodation.hotel'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Dreamlan Golf Hotel \\ Drimlend Otel', 'osm_id': 9930407478, 'tourism': 'hotel', 'osm_type': 'n'}}, 'distance': 3056, 'place_id': '51db81df2b5f0c494059d6a75f32443

...

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Hearthstone Inn Boutique Hotel Dartmouth/Halifax', 'country': 'Canada', 'country_code': 'ca', 'state': 'Nova Scotia', 'county': 'Halifax County', 'city': 'Dartmouth', 'municipality': 'Halifax Regional Municipality', 'postcode': 'B2Y 1N3', 'suburb': 'Grahams Grove', 'street': 'Prince Albert Road', 'housenumber': '313', 'lon': -63.5496598, 'lat': 44.6788064, 'state_code': 'NS', 'formatted': 'Hearthstone Inn Boutique Hotel Dartmouth/Halifax, 313 Prince Albert Road, Dartmouth, NS B2Y 1N3, Canada', 'address_line1': 'Hearthstone Inn Boutique Hotel Dartmouth/Halifax', 'address_line2': '313 Prince Albert Road, Dartmouth, NS B2Y 1N3, Canada', 'categories': ['accommodation', 'accommodation.hotel'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Hear

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Hotel History', 'country': 'Italy', 'country_code': 'it', 'state': 'Sicily', 'county': 'Trapani', 'city': 'Valderice', 'village': 'Crocevie', 'postcode': '91019', 'suburb': 'Paparella', 'street': 'Via Vespri', 'lon': 12.6289728, 'lat': 38.0437601, 'formatted': 'Hotel History, Via Vespri, 91019 Valderice TP, Italy', 'address_line1': 'Hotel History', 'address_line2': 'Via Vespri, 91019 Valderice TP, Italy', 'categories': ['accommodation', 'accommodation.hotel'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Hotel History', 'osm_id': 463676652, 'tourism': 'hotel', 'osm_type': 'n'}}, 'distance': 1284, 'place_id': '51a8250cb90842294059cf2853ee99054340f00103f901ec24a31b0000000092030d486f74656c20486973746f7279'}, 'geometry': {'type': 'Point', 'c

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Comfort Hotel Fosna', 'country': 'Norway', 'country_code': 'no', 'county': 'Møre og Romsdal', 'city': 'Kristiansund', 'municipality': 'Kristiansund', 'postcode': '6509', 'suburb': 'Innlandet', 'quarter': 'Allanenga', 'street': 'Hauggata', 'lon': 7.7311012, 'lat': 63.1122218, 'formatted': 'Comfort Hotel Fosna, Hauggata, 6509 Kristiansund, Norway', 'address_line1': 'Comfort Hotel Fosna', 'address_line2': 'Hauggata, 6509 Kristiansund, Norway', 'categories': ['accommodation', 'accommodation.hotel'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Comfort Hotel Fosna', 'osm_id': 253423923, 'tourism': 'hotel', 'osm_type': 'n'}}, 'distance': 92, 'place_id': '51f34300cba5ec1e40591165b2485d8e4f40f00103f90133f11a0f00000000920313436f6d666f727420486f74

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'country': 'Russia', 'country_code': 'ru', 'region': 'Siberian Federal District', 'state': 'Altai Krai', 'county': 'Mamontovsky District', 'city': 'Mamontovo', 'municipality': 'Мамонтовский сельсовет', 'postcode': '658560', 'street': 'Кашировская улица', 'housenumber': '18', 'lon': 81.62342032143351, 'lat': 52.7036176, 'formatted': 'Кашировская улица 18, Mamontovo, Altai Krai, Russia, 658560', 'address_line1': 'Кашировская улица 18', 'address_line2': 'Mamontovo, Altai Krai, Russia, 658560', 'categories': ['accommodation', 'accommodation.hotel', 'building', 'building.accommodation'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'osm_id': 427837660, 'tourism': 'hotel', 'building': 'yes', 'osm_type': 'w', 'addr:city': 'Мамонтово', 'addr:street': 'Кашировская

,City,Country,Lat,Lng,Humidity,Hotel Name
12,port elizabeth,ZA,-33.9180,25.5701,73,Waterford Hotel
30,bethel,US,41.3712,-73.4140,67,Hampton Inn Danbury
44,bredasdorp,ZA,-34.5322,20.0403,83,Victoria Hotel
48,anadyr,RU,64.7500,177.4833,93,Гостевой дом
52,albany,US,42.6001,-73.9662,62,Tru
...,...,...,...,...,...,...
591,colonia,DE,50.9333,6.9500,75,Wasserturm Hotel Cologne
592,cole harbour,CA,44.6724,-63.4751,80,Hearthstone Inn Boutique Hotel Dartmouth/Halifax
593,valderice,IT,38.0401,12.6151,74,Hotel History
596,kristiansund,NO,63.1115,7.7320,87,Comfort Hotel Fosna


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
# Configure the map plot

map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    scale=0.5,
    color="City",
    hover_cols=["City", "Country", "Hotel Name"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)